# Q-learning first attempt

Optionally install what you need:

In [1]:
# import sys
# !{sys.executable} -m pip install seaborn gym-retro Pillow keras tensorflow opencv-python pandas matplotlib scipy

First we set up all of the imports that we will have with the project, there are a bunch:

In [2]:
import random
import math
import retro
from PIL import Image
import gym
import pickle
import operator
import pprint
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import cv2 #OpenCV
import time
import gym_remote.client as grc
import gym_remote.exceptions as gre
import os
import json
import pandas as pd
from IPython.display import clear_output
from collections import deque
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (30, 30)
import seaborn as sns

from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf


KeyboardInterrupt: 

Helper functions handle disc IO, and help set the stage for recor keeping in a run. `show_img` was useful in rendering a frame of sonic, at various stages in the image processing. 

In [ ]:
# file system interactors
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f: #dump files into objects folder
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        if name == 'epsilon':
            return .7;
        return []
    else:
        return []
    
output_dir = './run-'+time.strftime("%Y%m%d-%H:%M")+'/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
loss_file_path = output_dir+"loss_df.csv"

#Intialize log structures from file if exists else create new#Intiali 
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])


In [ ]:
def show_img(image,graphs = False):
    """
    Show images in new window
    """
    while True:
#         print(image.shape)
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        processed = process_img(image)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)  
        cv2.moveWindow(window_title, 20,20);
#         imS = cv2.resize(screen, (800, 400)) 
#         cv2.imshow(window_title, screen)
        cv2.imshow(window_title, processed)
        cv2.waitKey(5)
        cv2.destroyAllWindows
        break


Set up a similar tracked environment as the jerk agent, but with a bit of extra stuff tacked in

In [ ]:
class TrackedEnv(gym.Wrapper):
    """
    An environment that tracks the current trajectory and
    the total number of timesteps ever taken.
    """

    def __init__(self, env):
        super(TrackedEnv, self).__init__(env)
        self.action_history = []
        self.reward_history = []
        self.total_reward = 0
        self.total_steps_ever = 0
        record_file_path = output_dir+"record.csv"
        self.record = pd.read_csv(record_file_path) if os.path.isfile(record_file_path) else pd.DataFrame(columns = ['Timesteps','Total_Score'])
        actions_file_path = output_dir+"actions.csv"
        self.actions = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['Action','Intention'])
        
    def best_sequence(self):
        """
        Get the prefix of the trajectory with the best
        cumulative reward.
        """
        max_cumulative = max(self.reward_history)
        for i, rew in enumerate(self.reward_history):
            if rew == max_cumulative:
                return self.action_history[:i + 1]
        raise RuntimeError('unreachable')

    # pylint: disable=E0202
    def reset(self, **kwargs):
        self.action_history = []
        self.reward_history = []
        self.total_reward = 0
        return self.env.reset(**kwargs)

    def step(self, action):
        self.total_steps_ever += 1
        self.action_history.append(action.copy())
        obs, rew, done, info = self.env.step(action)
        if done:
            data = pd.DataFrame({'Timesteps':[len(self.action_history)], 'Total_Score': [self.total_reward]})
            self.record = self.record.append(data)
#             self.record.loc[len(loss_df)] = score
#             self.record.append([self.total_reward, len(self.action_history)])
            print('rip')
        self.total_reward += rew
        self.reward_history.append(self.total_reward)
        return obs, rew, done, info


Hyperparameters, many of these are copied from the dino learning paper, and also the flappy bird paper referenced within.

In [ ]:
EXPLOIT_BIAS = 0.25  # 0.;5
TOTAL_TIMESTEPS = int(1e6)

#game parameters
ACTIONS = 8 # possible actions
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 200000. # timesteps to observe before training
EXPLORE = 200000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 100000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAMERATE= 4 #how often to render
LEARNING_RATE = 1e-4
img_rows , img_cols = 120,84
img_channels = 4 #We stack 4 frames

In [ ]:
def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same',input_shape=(img_cols,img_rows,img_channels)))  #20*40*4
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mean_squared_logarithmic_error',optimizer=adam)
    print("We finish building the model")
    return model
# buildmodel().summary()

The actions and random move function are the commands we use to actually make actions on our state. Random move had the option to use a supplied value, for example when a move was predicted by the model. 

In [ ]:
# Possible actions
# ["B", "A", "MODE", "START", "UP", "DOWN", "LEFT", "RIGHT", "C", "Y", "X", "Z"]
actions = [
    [True, False, False, False, False, False, False, False, False, False, False, False],
    [True, False, False, False, False, False, False, True, False, False, False, False],
    [False, False, False, False, False, False, False, True, False, False, False, False],
    [False, False, False, False, False, True, False, False, False, False, False, False],
    [True, False, False, False, False, True, False, False, False, False, False, False],
    [False, False, False, False, False, False, False, False, False, False, False, False],
    [False, False, False, False, False, False, True, False, False, False, False, False],
    [True, False, False, False, False, False, True, False, False, False, False, False],
]

In [ ]:
def random_move(env, Choice=None):
    done = False
    if Choice is None:
        Choice = random.randint(1,len(actions))-1
    # no info variable in contest environment
    obs, rew, done, _ = env.step(actions[Choice])
    
    return rew, done, obs, Choice


`proccessObs` was used in image cropping and resizing, but was ultimately dropped. 

In [ ]:
def proccessObs(obs):
    window_size_x = 180
    window_size_y = 100
    window_offset_x = 140 #int((320 - window_size_x)/2)
    window_offset_y = 100 #int((224 - window_size_y)/2)
    # lm = Image.fromarray(np.array(obs[window_offset_y:(window_offset_y+window_size_y),window_offset_x:(window_offset_x+window_size_x)]))
    # lm.show()
    # input("Press Enter to continue...")
    return obs[window_offset_y:(window_offset_y+window_size_y),window_offset_x:(window_offset_x+window_size_x)].flatten().tostring()



`process_img` takes a screen from the evironment and applies the resizing, edge detection, and color correction.

In [ ]:
def process_img(image):
    #crop out the dino agent from the frame
    height, width = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     print(height)
#     print(width)
    image = image[round(height*.25):height,round(width*.25):width] #img[y:y+h, x:x+w] 
    image = cv2.resize(image, (0,0), fx = 0.5, fy = 0.5) 
#     height, width = image.shape[:2]
#     print(height)
#     print(width)
    image = cv2.Canny(image, threshold1 = 100, threshold2 = 200) #apply the canny edge detection
    return  image   

In [ ]:
if __name__ == '__main__':
    try:
        main()
    except gre.GymRemoteError as exc:
        print('exception', exc)


Finially, there a a function that can take the hardcoded folder name of a previous run, and show you the loss over time as well as the distribution in moves between the random & model predictions. 

In [ ]:
def show_plots():
    rundir = 'run-20180519-14:14'
    fig, axs = plt.subplots(ncols=1,nrows =3,figsize=(15,15))
    axs[0].set_title('Loss')
    axs[1].set_title('Game Score progress')
    loss_df = pd.read_csv("./"+rundir+"/loss_df.csv")#.set_yscale('log')
    scores_df = pd.read_csv("./"+rundir+"/records.csv")
    actions_df = pd.read_csv("./"+rundir+"/actions.csv")
    actions_df['Action'] = actions_df['Action'].astype('float') 
    loss_df['loss'] = loss_df['loss'].astype('float') 
    loss_df.plot(use_index=True,ax=axs[0]).set_yscale('log')



    sns.distplot(actions_df['Action'].loc[actions_df['Intention'] == 'Predicted'])
    sns.distplot(actions_df['Action'].loc[actions_df['Intention'] == 'Random'])
    scores_df.plot(ax=axs[1])
    imgg = fig.canvas.draw()
show_plots()